In [ ]:
import numpy as np

import numpy as np

# Initialize the weight matrix and the pruning mask with random values
size = (10, 10)
W = np.random.randn(*size)

# Calculate the inverse Hessian matrix via Cholesky decomposition
H = np.dot(W.T, W)
L = np.linalg.cholesky(H)
H_inv = np.linalg.inv(H)

# Define the block size for the weight matrix
blocksize = 2
lambda_val = 1.0

drow, dcol = W.shape
Q = np.zeros((drow, dcol))  # quantized output
E = np.zeros((drow, blocksize))  # block quantization errors


for i in range(0, dcol, blocksize):
    for j in range(i, min(i + blocksize, dcol)):
        scale, zero, maxq = find_params(W[:, j]), # find quantization parameters
        Q[:, j] = quant(W[:, j], scale, zero, maxq)  # quantize column
        E[:, j-i] = (W[:, j] - Q[:, j]) / H_inv[j, j]  # quantization error
        W[:, j:i+blocksize] -= np.outer(E[:, j-i], H_inv[j, j:i+blocksize])  # update weights in block
    
    W[:, i+blocksize:] -= np.dot(E, H_inv[i:i+blocksize, i+blocksize:])  # update all remaining weights
    

def quant(x, scale, zero, maxq):
    if maxq < 0:
        return (x > scale / 2).float() * scale + (x < zero / 2).float() * zero
    q = np.clip(np.round(x / scale) + zero, 0, maxq)
    return scale * (q - zero)

def find_params(x, bits=8):
  maxq = 2 ** bits - 1
  xmax = np.max(x)
  xmin = np.min(x)
  scale = (xmax - xmin) / maxq
  zero = np.round(-xmin / scale)
  return scale. zero, maxq

quantized_W = Q